<a href="https://colab.research.google.com/github/QinaL/CS143Labs/blob/main/mlg_keypose_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cd drive/MyDrive/MLG/MLG_code/ && pwd

/content/drive/MyDrive/MLG/MLG_code


###

In [3]:
import tensorflow as tf
tf.__version__

'2.15.0'

### Data Gen

In [4]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_ID, labels, batchsize=32, dim=(28,28), n_channels=1,
                 n_classes = 10, shuffle=True):
        self.dim = dim
        self.list_ID = list_ID
        self.labels = labels
        self.dim = dim
        self.batchsize = batchsize
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
        #return super().on_epoch_end()
        self.indexes = np.arange(len(self.list_ID))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batchsize:(index+1)*self.batchsize]

        list_ID_temps = [self.list_ID[k] for k in indexes]

        X,y = self.__generate_data(list_ID_temps)

        return X, y

    def __generate_data(self, list_ID_temps):
        X = np.empty((self.batchsize, *self.dim))
        y = np.empty((self.batchsize), dtype=int)

        for i, ID in enumerate(list_ID_temps):
            X[i, ] = np.load(ID + '.npy')#np.load('prac_data/' + ID + '.npy')

            y[i] = self.labels[ID]

        return X, y#keras.utils.to_categorical(y, num_classes = self.n_classes)

    def __len__(self):
        return int(np.floor(len(self.list_ID)/self.batchsize))




#### Data

In [ ]:
import os, re, csv
import pandas as pd
import numpy as np

pwd = 'drive/MyDrive/MLG/MLG_code/fflipper_clips/'

clips = os.listdir(pwd)
print(len(clips))

x_data_all = []

x_ids = []
labels = {}
conv = {'in': 0, 'out': 1, 'up': 2, 'down': 3, 'left': 4, 'right': 5, 'upanddown': 6, 'leftandright':7, 'inandout':8}

for i in range(len(clips)):
  csv = open(pwd+clips[i])
  data = csv.readlines()
  l = len(data)
  keyposes = [list(map(float, data[1].strip().split(','))),
              list(map(float, data[l//2].strip().split(','))),
              list(map(float, data[l-1].strip().split(',')))
  ]

  # keyposes_select = [x[6:24]+x[75:] for x in keyposes]
  # for i in range(3):
  #    del keyposes_select[i][2::3]
  #    print(len(keyposes_select[i]))

  # print(clips[i], len(data[0]), len(data[1]), len(data[l-1]))
  # for the generator
  clip_id = clips[i][:-4]
  x_ids.append(clip_id)
  np.save(clip_id, keyposes)
  # np.save(clip_id, keyposes_select)
  clip_label = re.search(r'[a-zA-Z]+', clip_id).group()
  labels[clip_id] = conv[clip_label.lower()]

  # for use later
  x_data_all

  csv.close()

print(x_ids)
print(len(x_ids))
print(labels)
print(len(labels))

# 3 rows of 189 points for one clip



2165


#### Model Basic

In [ ]:
params = {'batchsize': 64, 'dim': (3,189), 'n_channels': 1,
                 'n_classes' : 9, 'shuffle':True }

training_generator = DataGenerator(x_ids, labels, **params)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(3, 189)),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    # tf.keras.layers.Dropout(0.02),
    tf.keras.layers.Dense(9)
])

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

model.compile(optimizer="adam", loss=loss_fn, metrics=['accuracy'])


#model.fit_generator(generator = training_generator, epochs = 5)

model.fit(training_generator, epochs = 10)

Epoch 1/10
33/33 [==============================] - 3s 19ms/step - loss: 1.7711 - accuracy: 0.2997
Epoch 2/10
33/33 [==============================] - 1s 19ms/step - loss: 1.3060 - accuracy: 0.5322
Epoch 3/10
33/33 [==============================] - 1s 19ms/step - loss: 1.0294 - accuracy: 0.6302
Epoch 4/10
33/33 [==============================] - 1s 20ms/step - loss: 0.8464 - accuracy: 0.7093
Epoch 5/10
33/33 [==============================] - 1s 20ms/step - loss: 0.7558 - accuracy: 0.7514
Epoch 6/10
33/33 [==============================] - 1s 19ms/step - loss: 0.6835 - accuracy: 0.7741
Epoch 7/10
33/33 [==============================] - 1s 24ms/step - loss: 0.6579 - accuracy: 0.7798
Epoch 8/10
33/33 [==============================] - 1s 28ms/step - loss: 0.6069 - accuracy: 0.8063
Epoch 9/10
33/33 [==============================] - 1s 32ms/step - loss: 0.5702 - accuracy: 0.8196
Epoch 10/10
33/33 [==============================] - 1s 30ms/step - loss: 0.5575 - accuracy: 0.8016


Separate into Test Sets And Model Evaluate

In [ ]:
labels_counts = [0, 0, 0, 0, 0, 0, 0, 0, 0]

for (clip, axis) in labels.items():
  #print(clip, axis)
  labels_counts[axis]+=1

#{'in': 0, 'out': 1, 'up': 2, 'down': 3, 'left': 4, 'right': 5,
# 'upanddown': 6, 'leftandright':7, 'inandout':8}

labels_counts

[388, 326, 334, 302, 368, 373, 24, 34, 16]

In [ ]:
cutoff = int(0.7*2165) #1515
cutoff1 = 1800
train_x_ids = x_ids[:cutoff]
# valid_x_ids = x_ids[cutoff:cutoff1]
test_x_ids = x_ids[cutoff:]

In [ ]:
training_generator = DataGenerator(train_x_ids, labels, **params)
# validation_generator = DataGenerator(valid_x_ids, labels, **params)

model.compile(optimizer="adam", loss=loss_fn, metrics=['accuracy'])
model.fit(training_generator, epochs = 10)


ValueError: could not broadcast input array from shape (3,132) into shape (3,189)

In [ ]:
x_test = []
y_test = []

for i in test_x_ids:
  x_test.append(np.load(i+'.npy'))
  y_test.append(labels[i])
  #print(i)

In [ ]:
model.evaluate(np.array(x_test), np.array(y_test), verbose =1)

21/21 [==============================] - 0s 2ms/step - loss: 0.4922 - accuracy: 0.8415


[0.4921935498714447, 0.8415384888648987]

Labels Distribution

In [ ]:
labels_counts_all = [0, 0, 0, 0, 0, 0, 0, 0, 0]
labels_counts_train = [0, 0, 0, 0, 0, 0, 0, 0, 0]
labels_counts_test = [0, 0, 0, 0, 0, 0, 0, 0, 0]

for i in test_x_ids:
  y = labels[i]
  labels_counts_all[y] +=1
  labels_counts_test[y] += 1

for i in train_x_ids:
  y = labels[i]
  labels_counts_all[y] +=1
  labels_counts_train[y] += 1

print("in, out, up, down, left, right, upanddown, leftandright, inandout")
print("all:", labels_counts_all)
print("train:", labels_counts_train)
print("test:", labels_counts_test)

in, out, up, down, left, right, upanddown, leftandright, inandout
all: [388, 326, 334, 302, 368, 373, 24, 34, 16]
train: [260, 231, 222, 207, 255, 266, 24, 34, 16]
test: [128, 95, 112, 95, 113, 107, 0, 0, 0]


In [ ]:
model.predict([x_test[0].tolist()])




1/1 [==============================] - 0s 141ms/step


array([[ 2.438324  ,  2.323279  , -1.3427991 , -0.24428414,  0.40739217,
         0.48151043, -2.63612   , -2.9101057 , -2.4040442 ]],
      dtype=float32)

Probabilities

In [ ]:
prob_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

prob_model.predict([x_test[0].tolist()])

1/1 [==============================] - 0s 70ms/step


array([[9.9999797e-01, 8.8972229e-08, 2.5569942e-09, 9.6002797e-09,
        2.4745589e-07, 1.8578648e-07, 1.1556996e-07, 4.6206252e-09,
        1.2889461e-06]], dtype=float32)

In [ ]:
model_with_prob = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(3, 189)),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    # tf.keras.layers.Dropout(0.02),
    tf.keras.layers.Dense(9),
    tf.keras.layers.Softmax()
])

loss_fn_2 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [ ]:
model_with_prob.compile(optimizer="adam", loss=loss_fn_2, metrics=['accuracy'])
model_with_prob.fit(training_generator, epochs = 10)

Epoch 1/10
23/23 [==============================] - 2s 19ms/step - loss: 1.8376 - accuracy: 0.2758
Epoch 2/10
23/23 [==============================] - 0s 19ms/step - loss: 1.4371 - accuracy: 0.5204
Epoch 3/10
23/23 [==============================] - 0s 19ms/step - loss: 1.1796 - accuracy: 0.5815
Epoch 4/10
23/23 [==============================] - 0s 20ms/step - loss: 0.9876 - accuracy: 0.6739
Epoch 5/10
23/23 [==============================] - 0s 20ms/step - loss: 0.8967 - accuracy: 0.7038
Epoch 6/10
23/23 [==============================] - 0s 19ms/step - loss: 0.8321 - accuracy: 0.7242
Epoch 7/10
23/23 [==============================] - 0s 19ms/step - loss: 0.7881 - accuracy: 0.7364
Epoch 8/10
23/23 [==============================] - 0s 19ms/step - loss: 0.7605 - accuracy: 0.7181
Epoch 9/10
23/23 [==============================] - 0s 20ms/step - loss: 0.7334 - accuracy: 0.7439
Epoch 10/10
23/23 [==============================] - 0s 18ms/step - loss: 0.6584 - accuracy: 0.7765


In [ ]:
model_with_prob.evaluate(np.array(x_test), np.array(y_test), verbose =1)

21/21 [==============================] - 0s 3ms/step - loss: 0.5450 - accuracy: 0.7969


[0.5450447201728821, 0.7969231009483337]

In [ ]:
model_with_prob.predict([x_test[0].tolist()])

1/1 [==============================] - 0s 95ms/step


array([[0.6471133 , 0.15781   , 0.03554948, 0.01106995, 0.07153244,
        0.07166701, 0.00116935, 0.00217144, 0.00191696]], dtype=float32)

In [ ]:
y_test[0]

0

Select Points

In [ ]:
for i in range(len(clips)):
  csv = open(pwd+clips[i])
  data = csv.readlines()
  l = len(data)
  print(data[0])
  keyposes = [list(map(float, data[1].strip().split(','))),
              list(map(float, data[l//2].strip().split(','))),
              list(map(float, data[l-1].strip().split(',')))
  ]

  # print(clips[i], len(data[0]), len(data[1]), len(data[l-1]))
  # for the generator
  # clip_id = clips[i][:-4]
  # x_ids.append(clip_id)
  # np.save(clip_id, keyposes)
  # clip_label = re.search(r'[a-zA-Z]+', clip_id).group()
  # labels[clip_id] = conv[clip_label.lower()]

  # for use later
  # x_data_all
  titles = data[0].strip().split(',')
  print(titles)
  print(titles[6:24])
  print(titles[75:])
  print(titles.index("left_wrist_x"))
  new = titles[6:24]+titles[75:]
  print(len(new))
  print(new)
  del new[2::3]
  print(new)
  print(len(new))

  keyposes_select = [x[6:24]+x[75:] for x in keyposes]

  # for i in keyposes:
  #   print(i)
  #   print(len(i))
  # # print(keyposes)

  # for i in keyposes_select:
  #   print(i)
  #   print(len(i))

  csv.close()
  break

nose_x,nose_y,nose_c,neck_x,neck_y,neck_c,rshoulder_x,rshoulder_y,rshoulder_c,relbow_x,relbow_y,relbow_c,rwrist_x,rwrist_y,rwrist_c,lshoulder_x,lshoulder_y,lshoulder_c,lelbow_x,lelbow_y,lelbow_c,lwrist_x,lwrist_y,lwrist_c,midhip_x,midhip_y,midhip_c,rhip_x,rhip_y,rhip_c,rknee_x,rknee_y,rknee_c,rankle_x,rankle_y,rankle_c,lhip_x,lhip_y,lhip_c,lknee_x,lknee_y,lknee_c,lankle_x,lankle_y,lankle_c,reye_x,reye_y,reye_c,leye_x,leye_y,leye_c,rear_x,rear_y,rear_c,lear_x,lear_y,lear_c,lbigtoe_x,lbigtoe_y,lbigtoe_c,lsmalltoe_x,lsmalltoe_y,lsmalltoe_c,lheel_x,lheel_y,lheel_c,rbigtoe_x,rbigtoe_y,rbigtoe_c,rsmalltoe_x,rsmalltoe_y,rsmalltoe_c,rheel_x,rheel_y,rheel_c,left_wrist_x,left_wrist_y,left_wrist_c,left_thumb_cmc_x,left_thumb_cmc_y,left_thumb_cmc_c,left_thumb_mcp_x,left_thumb_mcp_y,left_thumb_mcp_c,left_thumb_ip_x,left_thumb_ip_y,left_thumb_ip_c,left_thumb_tip_x,left_thumb_tip_y,left_thumb_tip_c,left_index_mcp_x,left_index_mcp_y,left_index_mcp_c,left_index_pip_x,left_index_pip_y,left_index_pip_c,l

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(3, 88)),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dropout(0.02),
    tf.keras.layers.Dense(9)
])

params = {'batchsize': 64, 'dim': (3,88), 'n_channels': 1,
                 'n_classes' : 9, 'shuffle':True }

training_generator = DataGenerator(train_x_ids, labels, **params)

model.compile(optimizer="adam", loss=loss_fn, metrics=['accuracy'])
model.fit(training_generator, epochs = 10)

Epoch 1/10
23/23 [==============================] - 3s 20ms/step - loss: 0.1357 - accuracy: 1.0000
Epoch 2/10
23/23 [==============================] - 0s 19ms/step - loss: 2.8551e-06 - accuracy: 1.0000
Epoch 3/10
23/23 [==============================] - 0s 20ms/step - loss: 1.1632e-06 - accuracy: 1.0000
Epoch 4/10
23/23 [==============================] - 0s 19ms/step - loss: 5.8138e-07 - accuracy: 1.0000
Epoch 5/10
23/23 [==============================] - 0s 19ms/step - loss: 5.6858e-07 - accuracy: 1.0000
Epoch 6/10
23/23 [==============================] - 0s 20ms/step - loss: 1.4587e-06 - accuracy: 1.0000
Epoch 7/10
23/23 [==============================] - 0s 18ms/step - loss: 3.2313e-07 - accuracy: 1.0000
Epoch 8/10
23/23 [==============================] - 0s 19ms/step - loss: 4.1253e-07 - accuracy: 1.0000
Epoch 9/10
23/23 [==============================] - 0s 18ms/step - loss: 5.1943e-07 - accuracy: 1.0000
Epoch 10/10
23/23 [==============================] - 0s 20ms/step - loss: 1.0

In [ ]:
model.evaluate(np.array(x_test), np.array(y_test), verbose =1)

21/21 [==============================] - 0s 2ms/step - loss: 1.9073e-06 - accuracy: 1.0000


[1.9073471548836096e-06, 1.0]

In [ ]:
model.predict([x_test[0].tolist()])

1/1 [==============================] - 0s 67ms/step


array([[13.95454   , -2.2803986 , -5.829891  , -4.506932  , -1.2574911 ,
        -1.5441258 , -2.0188477 , -5.2381926 ,  0.39285648]],
      dtype=float32)